In [1]:
print ("Hi i am kohinoor"  )

Hi i am kohinoor


In [2]:
%pwd

'd:\\chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\chatbot'

In [5]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file(data='D:\chatbot\Data')

In [8]:
#extracted_data

In [9]:
#chunking operation
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 82


In [11]:
import sentence_transformers
print(sentence_transformers.__version__)


c:\Users\Dell\anaconda3\envs\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.4.0


In [12]:
#using embedding model
from langchain.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_huggingface_embeddings()


C:\Users\Dell\AppData\Local\Temp\ipykernel_24068\3164718556.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv
import os

dotenv_path = "D:\chatbot/.env"

loaded = load_dotenv(dotenv_path=dotenv_path)

print("Loaded:", loaded)  


Loaded: True


In [17]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print("API Key:", PINECONE_API_KEY)  # Should print your API key


API Key: pcsk_qWCSH_PazQaYjM34pQi9dgMsTsfxQ8fHPh7WEVSjPpNdfif8vNs8mPeE6sYswk9sknpaK


In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "chatbott"

pc.create_index(
    name=index_name, 
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

In [20]:
import os
os.environ ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
#existing index

from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [27]:
retriever= docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [25]:
retrieved_docs = retriever.invoke("What is the advantage of system analysis and design?")

In [28]:
retrieved_docs

[Document(id='03682b91-0021-4577-950b-78ff0d2b6111', metadata={'page': 0.0, 'page_label': '1', 'source': 'D:\\chatbot\\Data\\crypto.pdf'}, page_content='characterization of these three objectives in terms of req uirements and the definition of a loss of \nsecurity in each category:'),
 Document(id='11c2b859-def1-46ee-8e32-7e69be3c2108', metadata={'page': 0.0, 'page_label': '1', 'source': 'D:\\chatbot\\Data\\crypto.pdf'}, page_content='computer system data from those with malicious intentions. \n \nNetwork S ecurity: It  consists of the  policies and practices adopted to prevent and \nmonitor unauthorized access, misuse, modification, or denial of a  computer network  and \nnetwork-accessible resources. Network security involves the authorization of access to data in a \nnetwork, which is controlled by the network administrator. \n \nThis definition introduces three key objectives that are at the heart of computer security:'),
 Document(id='0a2eb653-02c3-41f6-b3e0-3b90380be634', metadat

In [29]:
from langchain_community.llms import CTransformers
llm = CTransformers(
    model="D:\chatbot\model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={"max_new_tokens": 512, "temperature": 0.8}
)



In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-asnwering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),

    ]
)

In [31]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [32]:
response= rag_chain.invoke({"input": "what is the advantage of system analysis and design?"})
print(response["answer"])


Assistant: System analysis and design provide several advantages, including:

1. Improved system functionality: By analyzing and designing a system, we can identify and address any potential issues or limitations, ensuring that the system performs its intended function effectively and efficiently.
2. Increased security: A well-designed system is more likely to be secure and resistant to unauthorized access or manipulation, protecting sensitive data and systems from potential threats.
3. Better user experience: By considering the needs and behaviors of users, we can create a system that is intuitive, easy to use, and meets their expectations, leading to increased productivity and satisfaction.
